In [ ]:
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from datetime import timedelta
import datetime

In [ ]:
df1 = pd.read_csv(r'D:\DATAFLOW 2025\train.csv')

X_train = df1.iloc[:, 3:13].values
y_train = df1.iloc[:, 3:13].values

df2 = pd.read_csv(r'D:\DATAFLOW 2025\test.csv')

X_test = df2.iloc[:, 3:13].values
y_test = df2.iloc[:, 3:13].values

In [ ]:
print(df1.head())
print(df2.head())

   ProductID        Date    Zip  Units    Revenue          COGS
0       1228  2013-07-31  78773     12   19648.44  12309.747660
1        781  2014-03-12  90501     16   20351.52  13497.128064
2       2090  2013-11-29  44460     26  111367.62  91488.499830
3       1116  2018-08-29  30122     12   36280.44  21967.806420
4        993  2013-04-27  75789     14   55557.18  48645.866808
   ProductID        Date    Zip  Units    Revenue           COGS
0       1989  2022-06-01  54082     12   65764.44   65159.407152
1        582  2022-03-22  30012     14   59967.18   44705.532690
2       1828  2021-09-11  11377     15   51965.55   46665.063900
3        582  2022-06-01  55128     12   51400.44   38319.028020
4        415  2021-10-11  32174     29  333409.23  312271.084818


In [ ]:
print(f'{df1.shape[0]}')
print(f'{df2.shape[0]}')

901561
74682


In [ ]:
#Xử lí các dữ liệu bị lặp

duplicates = df1.duplicated().sum()
print(f'Tổng số dữ liệu bị lặp trong train.csv: {duplicates}')
if duplicates > 0:
    df = df1.drop_duplicates()
    print('Đã xử lí các dữ liệu bị lặp.')

duplicates= df2.duplicated().sum()
print(f'Tổng số dữ liệu bị lặp trong test.csv: {duplicates}')
if duplicates > 0:
    df = df2.drop_duplicates()
    print('Đã xử lí các dữ liệu bị lặp.')

Tổng số dữ liệu bị lặp trong train.csv: 36
Đã xử lí các dữ liệu bị lặp.
Tổng số dữ liệu bị lặp trong test.csv: 2
Đã xử lí các dữ liệu bị lặp.


In [ ]:
#Xử lí các dữ liệu thiếu

missing_values = df1.isnull().sum()
print(f'Tổng số các dữ liệu bị thiếu trong train.csv: \n{missing_values}')

missing_values = df2.isnull().sum()
print(f'Tổng số các dữ liệu bị thiếu trong test.csv: \n{missing_values}')
print(missing_values[missing_values > 0])


Tổng số các dữ liệu bị thiếu trong train.csv: 
ProductID    0
Date         0
Zip          0
Units        0
Revenue      0
COGS         0
dtype: int64
Tổng số các dữ liệu bị thiếu trong test.csv: 
ProductID     0
Date          0
Zip           0
Units         0
Revenue      41
COGS          0
dtype: int64
Revenue    41
dtype: int64


In [ ]:
df1_clean = df1.dropna()
df2_clean = df2.dropna()

missing_values_train_clean = df1_clean.isnull().sum()
print(f'Tổng số các dữ liệu bị thiếu trong train.csv sau khi loại bỏ: \n{missing_values_train_clean}')
print(missing_values_train_clean[missing_values_train_clean > 0])

missing_values_test_clean = df2_clean.isnull().sum()
print(f'Tổng số các dữ liệu bị thiếu trong test.csv sau khi loại bỏ: \n{missing_values_test_clean}')
print(missing_values_test_clean[missing_values_test_clean > 0])

Tổng số các dữ liệu bị thiếu trong train.csv sau khi loại bỏ: 
ProductID    0
Date         0
Zip          0
Units        0
Revenue      0
COGS         0
dtype: int64
Series([], dtype: int64)
Tổng số các dữ liệu bị thiếu trong test.csv sau khi loại bỏ: 
ProductID    0
Date         0
Zip          0
Units        0
Revenue      0
COGS         0
dtype: int64
Series([], dtype: int64)


In [ ]:
# Sắp xếp lại thứ tự cột 'Date' trong train.csv
df1['Date'] = pd.to_datetime(df1['Date'])
df1 = df1.sort_values(by='Date', ascending=True)
print(df1.head())


        ProductID       Date    Zip  Units   Revenue         COGS
188254        631 2010-07-04  50801      1   4280.85  4393.864440
301577       1041 2010-07-04  11777      1  11651.85  5941.278315
284103       1484 2010-07-04  15025      1    849.24   900.279324
66726         396 2010-07-04  94566      1  10769.85  9854.412750
69900         396 2010-07-04  27614      1  10706.85  9796.767750


In [ ]:
print(df1.info())
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
Index: 901561 entries, 188254 to 297110
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   ProductID  901561 non-null  int64         
 1   Date       901561 non-null  datetime64[ns]
 2   Zip        901561 non-null  int64         
 3   Units      901561 non-null  int64         
 4   Revenue    901561 non-null  float64       
 5   COGS       901561 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 48.1 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ProductID  74682 non-null  int64  
 1   Date       74682 non-null  object 
 2   Zip        74682 non-null  int64  
 3   Units      74682 non-null  int64  
 4   Revenue    74641 non-null  float64
 5   COGS       74682 non-null  float64
dty

In [ ]:
categorical_columns = ['ProductID', 'Date', 'Zip','Units','COGS']
df1[categorical_columns] = df[categorical_columns].astype('category')
df2[categorical_columns] = df[categorical_columns].astype('category')

print(df1.info())
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
Index: 901561 entries, 188254 to 297110
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   ProductID  74680 non-null   category
 1   Date       74680 non-null   category
 2   Zip        74680 non-null   category
 3   Units      74680 non-null   category
 4   Revenue    901561 non-null  float64 
 5   COGS       74680 non-null   category
dtypes: category(5), float64(1)
memory usage: 22.3 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ProductID  74680 non-null  category
 1   Date       74680 non-null  category
 2   Zip        74680 non-null  category
 3   Units      74680 non-null  category
 4   Revenue    74641 non-null  float64 
 5   COGS       74680 non-null  category
dtypes: category(5), float64(1)
memory usage: 2.0 MB
None


In [ ]:
#Đảm bảo  y_test chỉ có một cột
y_test_single_column = y_test[:, 0] if y_test.ndim > 1 else y_test

label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y_test_single_column)
label_encoded_y = label_encoder.transform(y_test_single_column)

seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_test, label_encoded_y, test_size=test_size, random_state=seed)


In [ ]:
#Mã hóa

label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y_test)
label_encoded_y = label_encoder.transform(y_test)
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_test, label_encoded_y, test_size=test_size, random_state=seed)

so sánh giá trị đã dự đoán với file test
- tính sai số
- có bao nhiêu row được tính

In [ ]:
#Đảm bảo cột y_train và y_test nhị phân (check lại)
y_train_binary = (y_train == 1).astype(int)
y_test_binary = (y_test == 1).astype(int)


#Tạo DMatrix
xgb_train = xgb.DMatrix(X_train, y_train_binary, enable_categorical=True)
xgb_test = xgb.DMatrix(X_test, y_test_binary, enable_categorical=True)

#Tạo mô hình (không dùng logistic, không chạy theo binary, dùng regression)
n = 50
params = {
    'objective': 'reg:squarederror',
    'max_depth': 3,
    'learning_rate': 0.1,
}

model = xgb.train(params=params, dtrain=xgb_train, num_boost_round=n)

#Dự đoán
preds = model.predict(xgb_test)
preds = (preds > 0.5).astype(int)  # chuyển đổi xác suất thành dự đoán nhị phân (so sánh với ngưỡng 0.5 để check nhị phân)

#Tính toán độ chính xác
accuracy = accuracy_score(y_test_binary, preds)
print('Độ chính xác của mô hình là:', accuracy * 100)

Độ chính xác của mô hình là: 100.0


In [ ]:
#Chia dữ liệu train.csv thành tập huấn luyện và tập validation
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train_binary, test_size=0.2, random_state=42)

# Create DMatrix for training and validation sets
xgb_train_split = xgb.DMatrix(X_train_split, y_train_split, enable_categorical=True)
xgb_val = xgb.DMatrix(X_val, y_val, enable_categorical=True)

#Tạo các giá trị của model
params = {
    'objective': 'binary:logistic',
    'max_depth': 3,
    'learning_rate': 0.1,
}

#Huấn luyện model với validation setset
evals = [(xgb_train_split, 'train'), (xgb_val, 'eval')]
model = xgb.train(params=params, dtrain=xgb_train_split, num_boost_round=50, evals=evals, early_stopping_rounds=10)

#Dự đoán dựa trên tệp traintrain
preds = model.predict(xgb_test)
preds = (preds > 0.5).astype(int)  #Chuyển probabilities thành dự đoán nhị phân

#Tính độ chính xác
accuracy = accuracy_score(y_test_binary, preds)
print('\n Độ chính xác của mô hình là:', accuracy * 100)

[0]	train-logloss:0.13605	eval-logloss:0.13605
[1]	train-logloss:0.12040	eval-logloss:0.12040
[2]	train-logloss:0.10727	eval-logloss:0.10727
[3]	train-logloss:0.09592	eval-logloss:0.09592
[4]	train-logloss:0.08597	eval-logloss:0.08596
[5]	train-logloss:0.07717	eval-logloss:0.07717
[6]	train-logloss:0.06937	eval-logloss:0.06936
[7]	train-logloss:0.06241	eval-logloss:0.06241
[8]	train-logloss:0.05620	eval-logloss:0.05620
[9]	train-logloss:0.05064	eval-logloss:0.05064
[10]	train-logloss:0.04566	eval-logloss:0.04565
[11]	train-logloss:0.04119	eval-logloss:0.04118
[12]	train-logloss:0.03717	eval-logloss:0.03717
[13]	train-logloss:0.03356	eval-logloss:0.03356
[14]	train-logloss:0.03031	eval-logloss:0.03031
[15]	train-logloss:0.02739	eval-logloss:0.02738
[16]	train-logloss:0.02475	eval-logloss:0.02474
[17]	train-logloss:0.02237	eval-logloss:0.02237
[18]	train-logloss:0.02023	eval-logloss:0.02022
[19]	train-logloss:0.01830	eval-logloss:0.01829
[20]	train-logloss:0.01655	eval-logloss:0.01655
[2

In [ ]:
preds_binary = (preds > 0.5).astype(int)

# Tính toán R-Squared (R2)
r2 = r2_score(y_test_binary, preds_binary)
print('Chỉ số r2 của mô hình là:', r2)

# Tính toán phần trăm sai số trung bình tuyệt đối (MAPE)
mape = mean_absolute_percentage_error(y_test_binary, preds_binary)
print('Chỉ số MAPE của mô hình là:', mape)

# Calculate giá trị trung bình bình phương (RMSE)
rmse = np.sqrt(mean_squared_error(y_test_binary, preds_binary))
print('Chỉ số RMSE của mô hình là:', rmse)

Chỉ số r2 của mô hình là: 1.0
Chỉ số MAPE của mô hình là: 0.0
Chỉ số RMSE của mô hình là: 0.0


In [ ]:
df = pd.read_csv(r'D:\\DATAFLOW 2025\\train.csv')

# chuẩn bị dữ liệu để dự đoán trong tương lai
future_dates = pd.date_range(start=pd.to_datetime(df['Date']).max() + timedelta(days=1), periods=730)
future_data = pd.DataFrame({
    'Zip': np.random.choice(df['Zip'], size=730),
    'Units': np.random.randint(1, 44, size=730),
    'COGS': np.random.uniform(1000, 30000, size=730)
})

# mã hóa dữ liệu thành số nguyên
label_encoder = LabelEncoder()
label_encoder.fit(df['Zip'])
future_data['Zip'] = label_encoder.transform(future_data['Zip'])

# chuẩn bị dữ liệu huấn luyện
X_train = df[['Zip', 'Units', 'COGS']]
y_train = df['Revenue']
X_train['Zip'] = label_encoder.transform(X_train['Zip'])

#huấn luyện mô hình XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', max_depth=3, learning_rate=0.1, n_estimators=100)
model.fit(X_train, y_train)

#tạo Dmatrix
xgb_future = xgb.DMatrix(future_data)

#Dự đoán doanh thu trong tương lai
future_predictions = model.predict(future_data)

# Tạo DataFrame chứa kết quả dự đoán
future_results = pd.DataFrame({'Date': future_dates, 'Predicted_Revenue': future_predictions})

#In kết quả
print(future_results)

C:\Users\lia\AppData\Local\Temp\ipykernel_31256\372795846.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Zip'] = label_encoder.transform(X_train['Zip'])


          Date  Predicted_Revenue
0   2021-01-01       13739.832031
1   2021-01-02       80956.960938
2   2021-01-03      202578.468750
3   2021-01-04       90514.000000
4   2021-01-05       82426.460938
..         ...                ...
725 2022-12-27      107310.484375
726 2022-12-28      -26806.207031
727 2022-12-29       22320.431641
728 2022-12-30       51552.746094
729 2022-12-31        6006.018066

[730 rows x 2 columns]


In [ ]:
future_results.to_csv(r'D:\\DATAFLOW 2025\\future_predictions.csv', index=False)

In [ ]:
# Tính toán sai số theo 2 chỉ số: Mean Squared Error (MSE), Mean Absolute Error (MAE), và số lượng hàng được sử dụng cho so sánh
actual_revenue = df['Revenue']
predicted_revenue = model.predict(df[['Zip', 'Units', 'COGS']])

mse = mean_squared_error(actual_revenue, predicted_revenue)
mae = mean_absolute_error(actual_revenue, predicted_revenue)

num_rows = len(actual_revenue)

print(f'Mean Squared Error: {round(mse, 2)}')
print(f'Mean Absolute Error: {round(mae, 2)}')
print(f'Số lượng hàng được sử dụng cho so sánh: {round(num_rows, 2)}')

Mean Squared Error: 2346491.92
Mean Absolute Error: 852.51
Số lượng hàng được sử dụng cho so sánh: 901561
